In [ ]:
import scvi

import scanpy as sc
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import os
import torch
import sys
from datetime import datetime

from matplotlib import pyplot as plt
from datetime import datetime

scvi.settings.progress_bar_style = "tqdm"

# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#hpc figures
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # print multiple outputs per code cell (not just last)

In [ ]:
sc.set_figure_params(figsize=(4, 4), dpi=100, dpi_save=300)

In [ ]:
nCores = 8
sc.settings.n_jobs = nCores #nCores
scvi.settings.num_threads = nCores # nThreads for PyTorch

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    scvi.settings.seed = seed # scvi-tools seed
    os.environ["PYTHONHASHSEED"] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(123)

In [ ]:
!cd /scratch/gent/vo/000/gvo00027/projects/Single_Cell_Neuroblastoma/NBAtlas
os.getcwd()

In [ ]:
output_dir = "09a_scVI_MalignantZoom_NBAtlas/"
os.makedirs(output_dir, exist_ok=True)

output_tables = output_dir + "/Tables/"
os.makedirs(output_tables, exist_ok=True)

output_figures = output_dir + "/Figures/"
os.makedirs(output_figures, exist_ok=True)

In [ ]:
sc.settings.figdir = output_figures

In [ ]:
sc.settings.verbosity = 4

In [ ]:
# mem check
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
# get memory in bytes
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}
mem

# Import data

In [ ]:
adata = sc.read('02_Create_AnndataObj_NBAtlas/nb_adata_NBAtlas.h5ad')
adata

## Subset malignant NE cells

In [ ]:
malignant_cells = pd.read_csv("06c_post_scVI_TumorZoom_R_plots_NBAtlas/Tables/06c_post_scVI_TumorZoom_R_plots_covSampleCopyKAT_MalignantCells_NBAtlas.csv", 
                       index_col=0)
malignant_cells

In [ ]:
malignant_cells["x"].to_list()

In [ ]:
adata = adata[adata.obs_names.isin(malignant_cells["x"].to_list())]
adata

In [ ]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state (with all genes) in `.raw`

In [ ]:
adata

In [ ]:
# save adata before taking HVGs
sc.write(adata=adata, 
        filename = output_dir + 'MalignantZoom_adata_a_NoInt_norm_full_all_genes_NBAtlas.h5ad') #this is adata_full see below

In [ ]:
adata = sc.read(output_dir + "MalignantZoom_adata_a_NoInt_norm_full_all_genes_NBAtlas.h5ad")

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=5000, 
    subset=True,
    layer="counts",
    flavor="seurat",
    batch_key="Study" 
)
adata

In [ ]:
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata, n_pcs=50)

In [ ]:
sc.pp.neighbors(adata, n_pcs=20, n_neighbors=20)
sc.tl.umap(adata, min_dist=0.6)

In [ ]:
adata

In [ ]:
sc.pl.umap(
    adata,
    color = ["Study"],
    save = "09_UMAP_MalignantZoom_a_NoInt_Study_NBAtlas.png"
)

In [ ]:
sc.pl.umap(
    adata,
    color = ["Assay"],
    save = "09_UMAP_MalignantZoom_a_NoInt_Assay_NBAtlas.png"
)

In [ ]:
sc.pl.umap(
    adata,
    color = ["Sample"],
    save = "09_UMAP_MalignantZoom_a_NoInt_Sample_NBAtlas.png"
)

In [ ]:
adata.write_h5ad(output_dir + "MalignantZoom_adata_b_NoInt_5000hvg_NBAtlas.h5ad")

In [ ]:
adata = sc.read(output_dir + "MalignantZoom_adata_b_NoInt_5000hvg_NBAtlas.h5ad")

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer = "counts",
    batch_key = "Sample"
)
adata

In [ ]:
model_kwargs = {
    "encode_covariates": True,
    "deeply_inject_covariates": False, 
    "use_layer_norm": "both",
    "use_batch_norm": "none"
}

model = scvi.model.SCVI(adata,    
    n_layers=2 
    )

model

In [ ]:
# early_stopping_kwargs
trainer_kwargs = {
    "save_best_state_metric": "elbo", 
    "early_stopping_patience": 10, #quicker stopping
    "threshold": 0, 
    "reduce_lr_on_plateau": True, 
}

loss_kwargs = {
    "lr_patience": 8, 
    "lr_factor": 0.1   
}

In [ ]:
model.view_anndata_setup() #check model

In [ ]:
print("Start =", datetime.now().strftime("%H:%M:%S"))

model.train(max_epochs = 500, early_stopping=True)

print("End =", datetime.now().strftime("%H:%M:%S"))

In [ ]:
model.save(output_dir + "09a_scVI_MalignantZoom_model_covSample_2layers_NBAtlas/", overwrite=True)

In [ ]:
#reload
model = scvi.model.SCVI.load(dir_path=output_dir + "09a_scVI_MalignantZoom_model_covSample_2layers_NBAtlas/", adata=adata, use_gpu=True)

In [ ]:
#plot
plt.plot(model.history["elbo_train"], label="train") #elbo_train_set
plt.plot(model.history["elbo_validation"], label="test")
plt.title("Negative ELBO over training epochs")
plt.legend()

In [ ]:
adata.obsm['X_pca_NoInt'] = adata.obsm['X_pca'].copy()
adata.obsm['X_umap_NoInt'] = adata.obsm['X_umap'].copy()

In [ ]:
adata.obsm["X_scVI"] = model.get_latent_representation() #store in adata
denoised_genes = model.get_normalized_expression( # store denoised values
    library_size=10e4,
    n_samples=1 
)
adata.layers["scvi_normalized"] = denoised_genes
adata.layers["scvi_normalized"] = sp.sparse.csr_matrix(adata.layers["scvi_normalized"])

In [ ]:
sc.pp.neighbors(adata, n_pcs=15, n_neighbors=20, use_rep = "X_scVI")
sc.tl.umap(adata, min_dist = 0.6)

In [ ]:
sc.pl.umap(
    adata,
    color = ["Study"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_Study_NBAtlas.png"
)

In [ ]:
sc.pl.umap(
    adata,
    color = ["Assay"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_Assay_NBAtlas.png"
)

In [ ]:
sc.pl.umap(
    adata,
    color = ["Sample"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_Sample_NBAtlas.png"
)

In [ ]:
sc.tl.leiden(adata, key_added = "leiden_scVI_res0p5", resolution = 0.5)

In [ ]:
sc.pl.umap(
    adata,
    color = ["leiden_scVI_res0p5"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_LeidenClustRes0p5_NBAtlas.png"
)

In [ ]:
sc.tl.leiden(adata, key_added = "leiden_scVI_res1", resolution = 1)

In [ ]:
sc.pl.umap(
    adata,
    color = ["leiden_scVI_res1"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_LeidenClustRes1_NBAtlas.png"
)

In [ ]:
sc.tl.leiden(adata, key_added = "leiden_scVI_res2", resolution = 2)

In [ ]:
sc.pl.umap(
    adata,
    color = ["leiden_scVI_res2"],
    save = "09_UMAP_MalignantZoom_c_scVI_covSample_2layers_LeidenClustRes2_NBAtlas.png"
)

In [ ]:
# Save
adata.write_h5ad(output_dir + "MalignantZoom_adata_c_covSample_2layers_NBAtlas.h5ad")

In [ ]:
# Reload
adata = sc.read(output_dir + "MalignantZoom_adata_c_covSample_2layers_NBAtlas.h5ad")